# Understanding Killwebs

A killweb is best understood as a series of interconnected systems and components which work together to complete a mission. This concept is the evolution of the kill chain which is a single, linear path of connected components that are used to complete each task within a given mission. Killwebs aim to enhance mission planning by striving for joint capabilities that are more robust than a kill chain could offer.

For example, say a given mission has a multitude of sensors and weaponry available for use. Any given kill chain would only consider a single sensor, weapon pair that could be used to complete the mission. On the other hand, a killweb simultaneously models and visualizes all components, systems, and their interactions to analyze a given mission and its capabilities.

We consider two ways of modeling and visualizing killwebs: at the component level and at the system level. We will begin by discussing the component level analysis, followed by a discussion at the system level. Although it is possible to apply these concepts to other mission frameworks or scenarios, the remainder of this notebook will be specific to the exemplar.

# Exemplar 1: Long Range Strikes

The mission you are tasked with involves striking a moving target from a long distance.

It is important to note that there are many different ways of mission decomposition into multiple tasks. Different branches, organizations, or teams might use their own methods for the tasks required to complete a mission. We will focus on the F2T2EA decomposition framework. An acronym for Find, Fix, Track, Target, Engage, Assess, this represents the necessary tasks needed to complete a mission. This mission decomposition was chosen for its simplicity and popularity, but this research can be applied to any task-based mission decomposition. Now we have the mission goal and the means to decompose the mission into a series of manageable tasks. In creating your own example, this would be a good time to fulfill the first two requirements of creating a config file and populating a tasks directory.

With this mission decomposition framework in mind, we have to:
1. Find - Identify the target to be struck. Successful completion of this task identifies the target to be a mobile anti-aircraft missile launcher.
2. Fix - Determine the target's location. Successful completion of this task provides the exact coordinates of the anti-aircraft launcher.
3. Track - Continuously monitor the target's movement. This step is continuous and its success ensures the anti-aircraft launcher's location remains up to date.
4. Target - Select the device that will be used to strike the anti-aircraft launcher. Taking into account all parameters, success here will provide the weapon used for the strike.
5. Engage - Send the selected weapon in to strike the anti-aircraft launcher.
6. Assess - Evaluate the results of the strike by determining whether or not the anti-aircraft launcher was destroyed.

Given a collection of components and systems, we can begin by creating the killwebs before analyzing it for any valuable data.

### Importing Functionality

We begin by importing the necessary libraries and classes.

In [ ]:
import os
%matplotlib widget
import matplotlib.pyplot as plt
from mimik.killweb import Killweb

### Data Loading

Our killwebs are generated from a given configuration file. This file contains a series of components. Each component includes the task it is assigned to complete, the parameters for the probabilistic model, and a list of components that it is connected to. These files are read from to generate the outputs placed in the given output directory.

In [ ]:
config_file = os.path.join(os.getcwd(), "configs", "killweb_interconnected.json")

## Component Level Killwebs

At the component level, each available component has a single task to be achieved alongside a single probabilistic model for calculating the likelihood of successfully completing the associated task. These components can be on their own, but are often smaller pieces of larger, interconnected systems. One example of a component might be a sensor whose purpose is the complete the "Fix" task of F2T2EA, and the results of the sensor could work with 3 different components which satisfy the "Track" task.

This trail of thought led to the idea that component level killwebs can be represented by acyclic directional graphs. Each node represents a component with a given task, and each node will have outgoing edges to compatible components responsible for completing the next task.

The associated probabilistic model can be as simple or complex as necessary. It could be a simple distance equation with a probabilistic value of 0 or 1 if the target is within the given range, or it could be the confidence of an AI model focused on detecting a given target provided the current luminosity, weather, and time of day.

### Creating the Killweb

With the path to the configuration file, the killweb can now be generated. WIthin the killweb's creation, a component graph will also be created alongside a ComponentMetrics object. We will touch on each of these object, beginning with the ComponentGraph

### Using the ComponentGraph

Initializing a ComponentGraph object creates a depth first, directional graph where each node represents a component, and edges represent information exchange. You might notice the "center" component which can be thought of as either a sentinel node for the final components to point to or as the final node representing a completed mission.

In [ ]:
killweb = Killweb(working_dir='.', config_file=config_file, silent=True)
for node in killweb.component_graph.nodes:
    component = killweb.component_graph.nodes[node]["component"]
    print("Killweb has component: %s with associated task, %s." % (component.full_name, component.task.task_name))

Here we can see our whole list of components and their associated tasks. 19 different components are available for completing this missions 6 different tasks. You will also notice the 'center' node with task Other. That is a sentinel component that all components with the task 'Assess' point to.

### Visualizing the Component Graph with NetworkX

Given the component graph. We can utilize a collection of Python libraries to display the component based killweb. The first graph shown is generated by Networkx, and the second is generated by PyGraphViz using the Networkx graph as input. It is important to note that the first Networkx graph is more useful outside of Jupyter as this is just a static representation of a dynamic figure that displays information when each node is hovered.

In [ ]:
killweb.create_component_networkx_visualization(False);

### Analyzing the Component Graph with Respect to Our Mission

Here, we can see all of the general components that can be used within our mission. The associations of the tasks to node labels can be found below. The numbers found within the node label correspond to the associated system number.

* Radar - Find
* Sensor - Fix
* Track Algorithm - Track
* Equation - Target
* Missle - Engage
* Personnel - Assess

For the long range strike mission, there are 3 systems available for use, each with theoretical capability to execute the mission on its own. However, the components can also be compatible with the components of other systems. One example being Sensor 2, which is compatible with Track Algorithms 1, 2, and 3. Witholding the system information from each component allows for a graph analysis of raw compatibility of inputs, outputs, and transmissions.

### Analyzing Killweb Probability

With the component graph visualization, we can begin to perform analysis onto our mission capabilities. First, we will output all of the paths in the killweb. Then, we can calculate the probability of each of these paths. The following code will find and print each of these paths before sorting them by decreasing probability. The function called runs N iterations for each path given, and then takes the average probability of all N iterations.

### Getting All Paths Within the Killweb

In [ ]:
killweb.print_all_paths_in_killweb()

### Monte Carlo Simulation of Paths

In [ ]:
killweb.monte_carlo_on_paths(10000)
killweb.print_probabilities_of_paths()

The above information is extremely useful in determining mission capability as there can be some paths with moderately high odds of success alongside paths with extremely low odds of success.

* That being said, with the given components, we can see the paths most likely to succeed hovers around 20%.
* The only paths with a chance of success involve Track Algorithm 1 and Equation 1.
* Inversely, path involving Track Algorithm 2, Track Algorithm 3, Equation 2, Equation 3, Missle 2, or Personnel 2 have a success probability of 0.0. This indicates that some or all of these components are defunct, and improvements to these components could lead to much greater odds of success to the mission overall as well as more viable paths through the killweb. As the Track Algorithms are earlier in the path, they should be investigated first.

With this information in mind, we can dive deeper into some of these paths. First beginning with a path more likely to succeed, then a path with no chance of success.

In [ ]:
path_to_test = ["Radar_2", "Sensor_2", "Track Algorithm_1", "Equation_1", "Missle_1", "Personnel_1"]
killweb.print_proportion_complete(path_to_test)
killweb.print_average_number_of_successes(path_to_test)
killweb.plot_monte_carlo_distribution(path_to_test)

In [ ]:
path_to_test = ["Radar_2", "Sensor_2", "Track Algorithm_3", "Equation_3", "Missle_3", "Personnel_3"]
prop_complete = killweb.print_proportion_complete(path_to_test)
killweb.print_average_number_of_successes(path_to_test)
killweb.plot_monte_carlo_distribution(path_to_test)

### Analyzing Monte Carlo Simulation of Paths

In the first set of graphs, it appears that there is a somewhat graceful degredation throughout the path. There is a significant drops between Radar_2 and Sensor_2 due to the Sensor_2 having a success probability between 0.5 and 0.6. This degredation continues as more failures occur in components that are not guarenteed to succeed. It appears that Sensor_2 and Track_Algorithm_1 could use some improvements. Sensor_2 has a range of probabilities depending on the conditions, but the sensor is inoperable betweeen 30% and 40% of the time. Track_Algorithm_1 also has a static probability between 0.4 and 0.5. Ensuring Sensor_2 can operate in conditions it is currently inoperable, and improving Track Algorithm 1 could significantly boost not only the success probability of the currently observed kill chain, but the killweb as a whole for all the paths involving either of these components.

The second set of graphs tells a much different story. We observe the same initial performance with Radar_2 and Sensor_2, but there are no successful events for later components. These graphs signify a dead zone in the kill chain, and further investigation into other paths containing Track Algorithm 2, Track Algorithm 3, Equation 2, Equation 3, Missle 2, and Personnel 2, reveal these components to all be a part of said dead zone. With the given structure of the killweb, the first place to looks are Track Algorithms 2 and 3 as these components mark the start of the dead zone.

### Component Centrality

One other important metric for the component graph is the centraility of each node. As we are working with a directional graph, we can compute both the in and out centrality of each node, and sort them accordingly.

In [ ]:
centrality = killweb.calculate_node_centrality()

Above we can see the sorted list of components by both their in and out centrality. A higher node centrality implies a high number of components pointing to it; a component is compatible with a high number of components capable of completing the next or previous task in the killweb. Inversely, a lower node centrality implies fewer nodes pointing to it; the component is compatible with fewer components capable of the next or previous task. Components with a centrality of 0 are not included as that indicates a broken killweb or that they starting components. It is also important to note that while an increase of 1 anywhere of in-degree centrality corresponds to an equal increase somewhere of out-degree centrality and vice-versa. That logic does not necessarily mean that there is an increase of 1 in total number of viable paths within the killweb. The increase is felt throughout the entire killweb, potentially resulting in many more paths with their own probabilities.

* The Radar components each have an in-centrality of 0, so they are not included within the in-degree centrality list. Personnel are not included in the out-degree centrality for the same reason.
* From the list, we see that Track Algorithm 1 and Missle 3 have an in-centrality of 3. With this information, we can infer that these components can work with a variety of input types or they require the least amount of specification.
* Sensor 2 and Radar 1 have an out-centrality of 3. Similarly, these components might be able to produce output in a variety of formats or one that is more generalizable.
* Out of the previously identified "dead zone" within the killweb of Track Algorithm 2, Track Algorithm 3, Equation 2, Equation 3, Missle 2, and Personnel 2, we see that all but Track Algorithm 3 have an in-degree centrality of 1. This indicates that paths with these components must stem from the same place which further reinforces the need to investigate the Track Algorithms found at the start of the dead zone.

### Addining a new Track Algorithm

To find the cause of the dead zone, we can add a new component which we will call "Track Algorithm_4" that comes from Sensor_2 and points to Equation_2 and Equation_3. With this new theoretical component added, we can definitively conclude whether or not Track Algorithms 2 and 3 are the sole cause of the dead zone. First we will add the node and run a monte carlo simulation with the new paths, and then we will analyze one of the new paths available.

In [ ]:
new_component_dict = {"task": "Track", "task_arguments": {"alpha_track": 125, "beta_track": 7, "tau": 10}, "system_name": "System_4"}
killweb.add_new_component(component_name="Track Algorithm_4", to_components=["Equation_2", "Equation_3"], from_components=["Sensor_2"], component_attributes=new_component_dict)
killweb.monte_carlo_on_paths(1000)
killweb.print_probabilities_of_paths(selected_component="Track Algorithm_4")
path_to_test = ["Radar_2", "Sensor_2", "Track Algorithm_4", "Equation_2", "Missle_2", "Personnel_2"]
killweb.print_proportion_complete(path_to_test)
killweb.print_average_number_of_successes(path_to_test)
killweb.plot_monte_carlo_distribution(path_to_test)

These results clearly show that the cause of the original kill web's dead zone was both Track Algorithms 2 and 3. Now, with Track Algorithm_4 added, Equation_2; Equation_3; Missle_2; and Personnel_2 are included in paths with Track Algorithm_4 with a success probability greater than zero. In fact, the paths with Track Algorithm_4 are those with some of the highest success probabilities in the entire kill web with overall success nearly doubling from the previous maximum.

In [ ]:
killweb.add_new_edge(from_component_name="Track Algorithm_4", to_component_name="Equation_3")
killweb.monte_carlo_on_paths(1000)
killweb.print_probabilities_of_paths(selected_component="Equation_3")

Now we can see that by adding an edge in betweeen Track Algorithm_4 and Equation_3, we do observe a handful of paths with a probability greater than zero. Therefore, we can conclude that Track Algorithms 2 and 3 are defunct components that result in a dead zone of the kill web, and correcting them in favor of a more superior Track Algorithm component should be a high priority.